In [13]:
import wfdb

In [ ]:
record = wfdb.rdrecord('ECGPCG0001')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\mosta\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\mosta\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\mosta\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
    self.io_lo

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [6]:
fs = record.fs
number_of_channels = record.n_sig
channel_names = record.sig_name
channel_names

['ECG', 'PCG']

In [ ]:

plt.plot(record.p_signal[:1000,0])
plt.show()

array([ 0.07415302,  0.07415302,  0.07526309,  0.07526309,  0.07570712,
        0.07615115,  0.07570712,  0.07659518,  0.07726123,  0.07726123,
        0.07770526,  0.07859332,  0.07903735,  0.07903735,  0.07903735,
        0.07903735,  0.08014742,  0.08014742,  0.08059145,  0.08147951,
        0.08147951,  0.08103548,  0.08147951,  0.08192354,  0.08192354,
        0.08258959,  0.08303362,  0.08347765,  0.08347765,  0.08392168,
        0.08503175,  0.08503175,  0.08591981,  0.08680787,  0.08747392,
        0.08880601,  0.08947205,  0.08991608,  0.09080414,  0.09124817,
        0.09235825,  0.09280228,  0.09369034,  0.09480041,  0.09568847,
        0.09524444,  0.0961325 ,  0.09679855,  0.09679855,  0.09724258,
        0.09768661,  0.09857467,  0.09857467,  0.09857467,  0.09968474,
        0.0990187 ,  0.09968474,  0.09968474,  0.1005728 ,  0.10012877,
        0.1005728 ,  0.1005728 ,  0.1005728 ,  0.10101683,  0.10146086,
        0.10101683,  0.10146086,  0.10146086,  0.10257094,  0.10